# Working with InferenceData
This notebook is a short introduction how to work with InferenceData and the underlying `xarray` data structure.


First, we load all the data and run the model. For more detail on this, check e.g. [this notebook](https://docs.pymc.io/notebooks/multilevel_modeling.html).

In [10]:
import pymc3 as pm
import pandas as pd
import numpy as np
import arviz as az

# Import radon data
srrs2 = pd.read_csv(pm.get_data('srrs2.dat'), index_col=0, dtype={"zip": str, "stflips":str, "zipflag":str, "cntyflips":str})
srrs2.columns = srrs2.columns.map(str.strip)
# Restrict data to Minnesota
srrs_mn = srrs2[srrs2.state=='MN'].copy()

In [11]:
srrs_mn.head()

,state,state2,stfips,zip,region,typebldg,floor,room,basement,windoor,...,stoptm,startdt,stopdt,activity,pcterr,adjwt,dupflag,zipflag,cntyfips,county
idnum,,,,,,,,,,,,,,,,,,,,,
5081,MN,MN,27,55735,5,1,1,3,N,,...,930,12088,12288,2.2,9.7,1146.499190,1,0,1,AITKIN
5082,MN,MN,27,55748,5,1,0,4,Y,,...,1615,11888,12088,2.2,14.5,471.366223,0,0,1,AITKIN
5083,MN,MN,27,55748,5,1,0,4,Y,,...,1515,20288,21188,2.9,9.6,433.316718,0,0,1,AITKIN
5084,MN,MN,27,56469,5,1,0,4,Y,,...,1410,122987,123187,1.0,24.3,461.623670,0,0,1,AITKIN
5085,MN,MN,27,55011,3,1,0,4,Y,,...,600,12888,13088,3.1,13.8,433.316718,0,0,3,ANOKA


In [12]:
srrs_mn.county = srrs_mn.county.map(str.strip)
mn_counties = srrs_mn.county.unique()
n_counties = len(mn_counties)
county_lookup = dict(zip(mn_counties, range(len(mn_counties))))

In [13]:
county = srrs_mn['county_code'] = srrs_mn.county.replace(county_lookup).values
radon = srrs_mn.activity
srrs_mn['log_radon'] = log_radon = np.log(radon + 0.1).values
log_radon = srrs_mn.log_radon.values
floor = srrs_mn.floor.values

In [32]:
with pm.Model() as partial_pooling:
    
    # Priors
    mu_a = pm.Normal("mu_a", mu=0, sigma=100)
    sigma_a = pm.HalfCauchy("sigma_a", 5)
    mu_b = pm.Normal("mu_b", mu=0, sigma=100)
    sigma_b = pm.HalfCauchy("sigma_b", 5)
    
    # Random intercepts
    alpha = pm.Normal("alpha", mu=mu_a, sigma=sigma_a, shape=n_counties)
    # Random slopes
    beta = pm.Normal("beta", mu=mu_b, sigma=sigma_b, shape=n_counties)
    
    # Model error
    sigma_y = pm.HalfCauchy("sigma_y", 100)
    
    # Expected value
    y_hat = alpha[county] + beta[county]*floor
    
    # Data likelihood
    y = pm.Normal("y", mu=y_hat, sigma=sigma_y, observed=log_radon)
    
    # sample
    trace = pm.sample(1000, tune=1000, chains=2)
    
    prior = pm.sample_prior_predictive()
    posterior_predictive = pm.sample_posterior_predictive(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_y, beta, alpha, sigma_b, mu_b, sigma_a, mu_a]
Sampling 2 chains: 100%|██████████| 4000/4000 [00:40<00:00, 98.48draws/s] 
There were 240 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5948371175128105, but should be close to 0.8. Try to increase the number of tuning steps.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8882995629765993, but should be close to 0.8. Try to increase the number of tuning steps.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
100%|██████████| 2000/2000 [00:02<00:00, 906.01it/s]


We've fit a hierarchical model that predicts the log radon levels in Minnesota. There is one predictor variable `floor` which indicates if the measurement was taken in the basement (0) or the first floor (1). Our linear model thus consists of two parameters, `alpha` and `beta`, and since it is a hierarchical model, each parameter is fitted per county. The model furthermore contains a `sigma_y` parameter for the noise and four hyperprior parameters for `alpha` and `beta`.

From the model we obtain three data sets: the trace (that is, the samples from the posterior), the prior predictive sample and posterior predictive samples. We conveniently combine all three data sets in an InferenceData object:

In [33]:
radon_data = az.from_pymc3(
            trace=trace,
            prior=prior,
            posterior_predictive=posterior_predictive,
        )
radon_data

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> observed_data

Let's start examine the posterior data:

In [57]:
radon_data.posterior

<xarray.Dataset>
Dimensions:  (chain: 2, county: 85, draw: 1000)
Coordinates:
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
  * county   (county) object 'AITKIN' 'ANOKA' ... 'WRIGHT' 'YELLOW MEDICINE'
Data variables:
    mu_a     (chain, draw) float64 1.458 1.456 1.456 1.459 ... 1.555 1.455 1.493
    mu_b     (chain, draw) float64 -0.5473 -0.5467 -0.5467 ... -0.6857 -0.6708
    alpha    (chain, draw, county) float64 1.03 0.9346 1.356 ... 1.789 1.424
    beta     (chain, draw, county) float64 -0.5419 -0.521 ... -0.6657 -0.8501
    sigma_a  (chain, draw) float64 0.2567 0.2584 0.2584 ... 0.397 0.3989 0.371
    sigma_b  (chain, draw) float64 0.01609 0.01714 0.01714 ... 0.1255 0.1117
    sigma_y  (chain, draw) float64 0.7398 0.7392 0.7392 ... 0.7259 0.7041 0.7146
Attributes:
    created_at:                 2019-11-01T16:46:19.820119
    inference_library:          pymc3
    inference_library_version:  3.7

Under data variables, we see the parameters of our model, here, there are 7 parameters. After each parameter comes the dimensions of the parameter. 
By default, every parameter has always the `chain` and `draw` dimensions. For this model, we fitted two chains, each with 1000 draws.
Accessing the fifth draw from the first chain for the parameter `mu_a` for example would then look like this:

In [39]:
radon_data.posterior["mu_a"][0,5]

<xarray.DataArray 'mu_a' ()>
array(1.460706)
Coordinates:
    chain    int64 0
    draw     int64 5

Both `alpha` and `beta` each have a third dimension that here are only cryptically named `alpha_dim_0` and `beta_dim_0`. These are the county dimensions. For each of the 85 counties, we fit two chains with 1000 draws each. We can give these dimensions better names by declaring the dimensions when creating the InferenceData object:

In [44]:
radon_data = az.from_pymc3(
            trace=trace,
            prior=prior,
            posterior_predictive=posterior_predictive,
            # the additional dimension for alpha and beta are called 'county'
            dims={'alpha': ['county'], 'beta': ['county']},
            # the dimension called 'county' has the county names as coordinates
            coords={'county': mn_counties},
        )

In [65]:
posterior = radon_data.posterior
posterior

<xarray.Dataset>
Dimensions:  (chain: 2, county: 85, draw: 1000)
Coordinates:
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
  * county   (county) object 'AITKIN' 'ANOKA' ... 'WRIGHT' 'YELLOW MEDICINE'
Data variables:
    mu_a     (chain, draw) float64 1.458 1.456 1.456 1.459 ... 1.555 1.455 1.493
    mu_b     (chain, draw) float64 -0.5473 -0.5467 -0.5467 ... -0.6857 -0.6708
    alpha    (chain, draw, county) float64 1.03 0.9346 1.356 ... 1.789 1.424
    beta     (chain, draw, county) float64 -0.5419 -0.521 ... -0.6657 -0.8501
    sigma_a  (chain, draw) float64 0.2567 0.2584 0.2584 ... 0.397 0.3989 0.371
    sigma_b  (chain, draw) float64 0.01609 0.01714 0.01714 ... 0.1255 0.1117
    sigma_y  (chain, draw) float64 0.7398 0.7392 0.7392 ... 0.7259 0.7041 0.7146
Attributes:
    created_at:                 2019-11-01T16:46:19.820119
    inference_library:          pymc3
    inference_library_version:  3.7

We can now see that the third dimension of `alpha` and `beta` is actually the same dimension and by providing the county names, we can now access the samples by county name and don't first have to look up the integer value.
To access the `alpha` and `beta` samples for the county 'AITKIN', we can use

In [66]:
posterior[["alpha", "beta"]].sel(county = "AITKIN")

<xarray.Dataset>
Dimensions:  (chain: 2, draw: 1000)
Coordinates:
    county   <U6 'AITKIN'
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Data variables:
    alpha    (chain, draw) float64 1.03 1.067 1.067 1.069 ... 1.591 0.8588 1.625
    beta     (chain, draw) float64 -0.5419 -0.5409 -0.5409 ... -0.7239 -0.6897
Attributes:
    created_at:                 2019-11-01T16:46:19.820119
    inference_library:          pymc3
    inference_library_version:  3.7

Having the information about the chain and draw is especially useful when doing model diagnostics, but we might not care so much about the chain or draw number afterwards. Or, if we only fit one chain, we don't care about having the chain dimension with only one coordinate.
In that case, we can stack the chain and draw dimension to a new dimension that we call `samples`:

In [67]:
posterior = posterior.stack(samples = ["draw", "chain"])
posterior

<xarray.Dataset>
Dimensions:  (county: 85, samples: 2000)
Coordinates:
  * county   (county) object 'AITKIN' 'ANOKA' ... 'WRIGHT' 'YELLOW MEDICINE'
  * samples  (samples) MultiIndex
  - draw     (samples) int64 0 0 1 1 2 2 3 3 4 4 ... 10 11 11 12 12 13 13 14 14
  - chain    (samples) int64 0 1 0 1 0 1 0 1 0 1 0 1 ... 0 1 0 1 0 1 0 1 0 1 0 1
Data variables:
    mu_a     (samples) float64 1.458 1.603 1.456 1.557 ... 1.455 1.361 1.493
    mu_b     (samples) float64 -0.5473 -0.7468 -0.5467 ... -0.4412 -0.6708
    alpha    (county, samples) float64 1.03 0.9907 1.067 ... 1.364 1.267 1.424
    beta     (county, samples) float64 -0.5419 -0.7551 ... -0.4336 -0.8501
    sigma_a  (samples) float64 0.2567 0.3073 0.2584 ... 0.3989 0.3881 0.371
    sigma_b  (samples) float64 0.01609 0.1523 0.01714 ... 0.1255 0.4774 0.1117
    sigma_y  (samples) float64 0.7398 0.7221 0.7392 ... 0.7041 0.6609 0.7146
Attributes:
    created_at:                 2019-11-01T16:46:19.820119
    inference_library:          

Since the floor predictor variable can only take on values of 0 or 1, we know that `alpha` represents the log radon level in the basement floor.
Similarly, `alpha` + `beta` represents the log radon level in the first floor.
If we want to know the radon level at the first floor, we simply sum up the variables and exponentiate:

In [92]:
posterior["first_floor_radon"] = np.exp(posterior["alpha"] + posterior["beta"]*1)
posterior

<xarray.Dataset>
Dimensions:            (county: 85, samples: 2000)
Coordinates:
  * county             (county) object 'AITKIN' 'ANOKA' ... 'YELLOW MEDICINE'
  * samples            (samples) MultiIndex
  - draw               (samples) int64 0 0 1 1 2 2 3 3 ... 11 12 12 13 13 14 14
  - chain              (samples) int64 0 1 0 1 0 1 0 1 0 1 ... 1 0 1 0 1 0 1 0 1
Data variables:
    mu_a               (samples) float64 1.458 1.603 1.456 ... 1.455 1.361 1.493
    mu_b               (samples) float64 -0.5473 -0.7468 ... -0.4412 -0.6708
    alpha              (county, samples) float64 1.03 0.9907 ... 1.267 1.424
    beta               (county, samples) float64 -0.5419 -0.7551 ... -0.8501
    sigma_a            (samples) float64 0.2567 0.3073 0.2584 ... 0.3881 0.371
    sigma_b            (samples) float64 0.01609 0.1523 ... 0.4774 0.1117
    sigma_y            (samples) float64 0.7398 0.7221 0.7392 ... 0.6609 0.7146
    first_floor_radon  (county, samples) float64 1.63 1.266 ... 2.302 1.776

Similarly as with pandas Dataframes, we can use groupby and aggregation function on our xarray Dataset. To obtain the median radon level on the first floor for each county, we can proceed as follows:

In [73]:
posterior["first_floor_radon"].groupby("county").median(dim=["samples"])

<xarray.DataArray 'first_floor_radon' (county: 85)>
array([1.808428, 1.324706, 2.386905, 2.3313  , 2.382273, 2.369587, 3.905642,
       2.746497, 1.852786, 2.220663, 2.351363, 2.484625, 1.83248 , 3.353187,
       2.236875, 1.977339, 1.950442, 2.054268, 2.028813, 2.727675, 2.931446,
       1.502852, 2.403771, 3.41584 , 3.508225, 2.057655, 2.622938, 2.005074,
       2.115492, 1.703854, 3.194219, 2.048759, 2.799348, 2.48283 , 1.679012,
       3.798932, 1.318437, 2.955076, 2.827303, 3.253778, 3.259585, 2.27361 ,
       2.25055 , 1.841993, 1.974204, 2.139086, 1.875827, 1.901116, 2.5914  ,
       2.844377, 3.137511, 2.58782 , 2.243324, 1.899855, 2.721476, 2.022168,
       1.629826, 2.678833, 2.610007, 2.145379, 1.995357, 3.672248, 2.530163,
       3.073486, 2.418039, 2.738686, 3.167066, 1.788324, 2.213257, 1.403773,
       2.315641, 2.624639, 2.646808, 2.020126, 2.494738, 2.907658, 2.838268,
       2.167601, 1.672793, 1.993298, 3.817495, 2.50442 , 2.270436, 2.735169,
       2.315437])
Coordi

It is straight-forward to transform this to a pandas dataframe:

In [76]:
posterior["first_floor_radon"].groupby("county").median(dim=["samples"]).to_dataframe().reset_index()

,county,first_floor_radon
0,AITKIN,1.808428
1,ANOKA,1.324706
2,BECKER,2.386905
3,BELTRAMI,2.331300
4,BENTON,2.382273
...,...,...
80,WATONWAN,3.817495
81,WILKIN,2.504420
82,WINONA,2.270436
83,WRIGHT,2.735169
